<div>
<img src="./images/sunpy_logo.png" width="500" align="left"/>
</div>

# An introduction to SunPy for your heliophysics needs and use with Solar Orbiter and Aditya-L1!  

The `sunpy` Python package is a community-developed, free, and open-source solar data analysis environment for Python!

`sunpy` provides the core functionality and tools to analyze solar data with Python!

In this notebook we'll go through an introduction of how we can use sunpy and other scientific packages such as astropy etc to perform solar data analysis. We'll focus on how to search and query data, download data, load data into different containers, and how to plot and manipulate data.

We will show:
 1. Querying and downloading data within the sunpy ecosystem with Fido - including the SOAR!
 2. Data containers within sunpy (`Map` & `TimeSeries`)
 3. Coordinates framework


# 1. Downloading data within sunpy

## 1. 1 Overview of `Fido` Unified Downloader


* [`Fido`](https://docs.sunpy.org/en/stable/guide/acquiring_data/fido.html#fido-guide) is sunpy's interface for searching and downloading solar physics data.

* It offers a unified interface for searching and fetching data irrespective of the underlying client or webservice from where the data is obtained.

* Offers a way to search and accesses multiple instruments and all available data providers in a single query.

* It supplies a single, easy, consistent and *extendable* way to get most forms of solar physics data the community need 

Fido offers access to data available through:

 * **VSO**
 * **JSOC** (through `drms`)
 * **Individual data providers** from web accessible sources (http, ftp, etc)
 * **HEK**
 * **HELIO**
 
As described here `Fido` provides access to many sources of data through different clients, these clients can be defined inside sunpy or in other packages. Lets print the current list of available clients within sunpy.

In [ ]:
from sunpy.net import Fido, attrs as a
from sunpy.time import parse_time
import sunpy.map
import sunpy.timeseries
from sunpy.coordinates import frames, get_body_heliographic_stonyhurst, get_horizons_coord, transform_with_sun_center
import sunpy.data.sample


import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.visualization import AsymmetricPercentileInterval, ImageNormalize, LogStretch
from reproject import reproject_interp
from reproject.mosaicking import reproject_and_coadd
from astropy.wcs import WCS

import matplotlib.pyplot as plt
from matplotlib import colors
import glob
import numpy as np

In [ ]:
Fido

### Using attributes to search for data with Fido

Sunpy uses specified **attributes** to search for data using Fido. The range of these attributes is located in the `attrs` submodule. These `attr` parameters can be combined together to construct data search queries, such as searching over a certain time period, for data from a certain instrument with a certain wavelength etc.

Different clients and provides will have client-specific attributes, but the core attributes are:

* `a.Time`
* `a.Instrument`
* `a.Wavelength`


Lets look at how these attributes work in more detail.

First we can look at `a.Time`, which is used to specify the timerange of a query.

In [ ]:
a.Time("2022-03-28 11:00", "2022-03-28 14:00")

We can inspect the instrument attribute to see what instrument `attrs` are currently supported through sunpy. Here we can see the instrument name (i.e. the name to be passed to the `a.Instrument` attribute, the client from which the data is available to access, and the full name of the instrument.)

In [ ]:
a.Instrument

In [ ]:
a.Instrument.eit

To search for certain wavelengths, we need to specify the input as an `astropy.Quantity` which is a the combination of a value and an associated unit. This is something is universal in the sunpy stack - that every physical input/output is a `Quantity`.

In [ ]:
a.Wavelength(17.1*u.nm)

## 1.2. Constructing a search query
 ### A simple query

Lets create a simple query to search for data from AIA over a particular time period

In [ ]:
result = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                     a.Instrument("AIA"))

In [ ]:
result

Now lets make our query a bit more specific, say, say we only want one wavelength band from AIA. This can be achieved by specifying the `Wavelength` attribute within the search. The `a.Wavelength` attribute is passed as an `astropy.Quantity`:

In [ ]:
result = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(304*u.angstrom))

In [ ]:
result

In [ ]:
result = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 12:00"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(304*u.angstrom),
                     a.Sample(10*u.min))

In [ ]:
type(result)

## 1.3 Downloading the data

Now we can show how data that is queried above can be downloaded. Once the data you have searched for (and filtered etc) is constructed into a query using `Fido.search`, you can then easily download them using `Fido.fetch`.

The data is downloaded via asynchronous and parallel download streams (via parfive), and also allows for failed data downloads to be recognized so that files can be re-requested if not downloaded.

Lets now look at how a `UnifiedResponse` from a `Fido.search` can be passed to `Fido.fetch` to download the data§§§m

In [ ]:
files = Fido.fetch(result)

In [ ]:
files.errors

These files are downloaded to a local location set in the sunpy.config.file, which by default is ~/sunpy/data/. Fido.fetch returns a parfile.Results object which gives the path to where the files are downloaded to

In [ ]:
print(files[0])

You can also define what directory you want the files to be saved to by passing the directory path to the path keyword in Fido.fetch. For example, I want to download these files to a local directory `./AIA/<name_of_file>`

In [ ]:
Fido.fetch(result, path="./{instrument}/{file}")


## 1. 4 Downloading Solar Orbiter Data: sunpy-soar

Within `sunpy` core, we support a number of clients to common data providers. However, the `Fido` search interface is extensible such that external packages can write that their own clients that extend `Fido` in order to additional data sources. One such example is the `sunpy_soar` package which adds a client for the [Solar Orbiter Archive (SOAR)](https://soar.esac.esa.int/soar/).

In [ ]:
import sunpy_soar
from sunpy_soar.attrs import Product

Note that after importing `sunpy_soar`, the SOAR is now listed as a client that `Fido` will search.

In [ ]:
Fido

In [ ]:
eui_query = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                       a.Instrument.eui)

In [ ]:
eui_query

In [ ]:
eui_query = Fido.search(a.Time("2022-03-28 11:00", "2022-03-28 14:00"), 
                        a.soar.Product("EUI-FSI304-IMAGE"), 
                        a.Level(2))

In [ ]:
eui_query

In [ ]:
f = Fido.fetch(eui_query, path="./{instrument}/{file}")

In [ ]:
f[0]

We can also search for other data products, for example the Solar Orbiter MAG

In [ ]:
mag_query = Fido.search(a.Time("2022-02-15", "2022-02-18"), 
                        a.soar.Product("MAG-RTN-NORMAL-1-MINUTE"), 
                        a.Level(2))

In [ ]:
mag_query

In [ ]:
ff = Fido.fetch(mag_query, path="./")

# 2. Data Containers


Now we have seen how we can search for and download data - lets now look at how we can read this data in.

SunPy provides core data type classes that are designed to provide a consistent interface across data types (timeseries and images) as well as data sources from numerous instruments and observations. They handle all of the manipulation necessary to read data in from mission-specific files. The two main datatypes in SunPy are

1. `TimeSeries`  
2. `Map`

## 2.1 TimeSeries

The structure of a [`TimeSeries`](https://docs.sunpy.org/en/stable/guide/data_types/timeseries.html) consists of times and measurements and the underlying structure is that of a `pandas.DataFrame`. 

SunPy TimeSeries supports time-series data from a wide range of solar-focused instruments. `TimeSeries` can either be created manually or from source files that are currently supported. If a supported file is passed to `TimeSeries` it will automatically detect its source and its instrument-specific meta data will be loaded. 

Lets create a timeseries from out sample data which is X-ray flux from the GOES X-ray Sensor Data. This data file was downloaded locally in the steps previously! 

In [ ]:
xrs = sunpy.timeseries.TimeSeries(sunpy.data.sample.GOES_XRS_TIMESERIES)

In [ ]:
xrs

In [ ]:
xrs.plot()

### Inspect the `TimeSeries`

Lets now inspect the `TimeSeries` and get at the data. A `TimeSeries` holds data as well as meta data and unit data.

In [ ]:
xrs.meta

In [ ]:
xrs.units

In [ ]:
xrs.to_dataframe()

In [ ]:
xrs.to_table()

## Manipulating the timeseries data

We can manipulate the timeseries, such as truncating (slicing) the data over a certain time period

In [ ]:
xrs.truncate("2011-06-07 05:00", "2011-06-07 08:00").peek()

## Solar Orbiter timeseries example

You can also pass a list of files to timeseries, and uses the `concatenate` keyword to create one continous timeseries. 

In [ ]:
mag_files = glob.glob("*solo_L2_mag*.cdf")
mag_files.sort()


In [ ]:
mag_files

In [ ]:
mag_solo = sunpy.timeseries.TimeSeries(mag_files, concatenate=True)

In [ ]:
mag_solo.columns

In [ ]:
mag_solo.peek(['B_RTN_0', 'B_RTN_1', 'B_RTN_2'])

# 2.2 Map
The sunpy [`Map`](https://docs.sunpy.org/en/stable/guide/data_types/maps.html) class provides the data type structure to store 2-dimensional data associated with a coordinate system.  This allows users to store and manipulate images of the Sun and the heliosphere

Maps from all instruments are created using the `sunpy.map.Map` 'factory'. This class takes a wide variety of map-like inputs, for one or more maps and returns you one or many maps. All maps, irrespective of the instrument, behave the same and expose the same functions and properties, however, depending on the instrument different metadata might be read or corrections made.

In [ ]:
aia_files = glob.glob("./AIA/*.fits"); aia_files.sort()

In [ ]:
eui_files = glob.glob("./EUI/*.fits"); eui_files.sort()

In [ ]:
eui_files[2], aia_files[2]

In [ ]:
aia_map = sunpy.map.Map(aia_files[2])
eui_map = sunpy.map.Map(eui_files[2])

We can easily visualize a map after loading it using the quicklook functionality.

In [ ]:
aia_map

`Map` provides customized loaders for a number of different instruments, however, if the data file follows the FITS data standards for coordinate information etc then map should be able to read it by default.

## Attributes of Map

`Map` provides a common interface to most 2D imaging solar datasets and provides several useful pieces of metadata. As mentioned in the intro slide, `Map` is a container for holding your data and metadata (usually from the FITS header) together.

The `.meta` and `.data` attributes provide access to the metadata and underlying array of image data, respectively.



In [ ]:
aia_map.data

In [ ]:
aia_map.meta



However, this metadata can be terse, non-homogeneous, and sometimes difficult to parse. `Map` provides several attributes derived from the underlying raw metadata that expose a uniform interface to the metadata for each map.



In [ ]:
aia_map.wavelength

In [ ]:
aia_map.rsun_meters

### Coordinate Information

Each `Map` also exposes information about which coordinate system the image was taken in, including the location of the spacecraft that recorded that observation.

`sunpy` leverages and extends the powerful astropy coordinate framework that we heard about in the previous tutorial. Additionally, we'll talk more about the sunpy.coordinates subpackage in the next notebook and show some neat examples.

For each `Map`, we can easily access what coordinate frame the observation cooresponds to.

In [ ]:
aia_map.coordinate_frame

In [ ]:
aia_map.observer_coordinate

### `Map` and WCS (World Coordinate System)

The world coordinate system (WCS) formalizes provides us a framework for transforming between pixel and world coordinates. The functionality to deal with WCS within sunpy is from the `astropy` package.




In [ ]:
aia_map.wcs

In [ ]:
type(aia_map.wcs)


### World and Pixel Coordinates

We can convert between the world coordinates (arcsec) to pixel coordinates using the `world_to_pixel` method on map which takes a `SkyCoord` and then returns the pixel coordinate. Similarly we can find the world coordinate to the pixel (or array) index. This is done with the `pixel_to_world` method. Lets first look at finding the array (pixel) index for the center of the Sun (0, 0) arcsec:



In [ ]:
aia_map.world_to_pixel(SkyCoord(0*u.arcsec, 0*u.arcsec, frame=aia_map.coordinate_frame))

In [ ]:
aia_map.pixel_to_world(0*u.pix, 0*u.pix)

## Visualization of `Map`

### Plotting a map

In [ ]:
fig = plt.figure(figsize=(8, 8))
aia_map.plot(clip_interval=[1, 99.9]*u.percent)
aia_map.draw_limb()
aia_map.draw_grid(color='w')

In [ ]:
plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection=eui_map)
eui_map.plot()
eui_map.draw_grid(color='g')

In [ ]:
plt.figure(figsize=(8,8))
eui_map.plot(cmap="viridis")

## Inspecting and Manipulating the data

In [ ]:
aia_map.data.shape

In [ ]:
print("Mean:", aia_map.mean(), "Max:", aia_map.max(), "Min:", aia_map.min(),  "Std:", aia_map.std())

### Rotate a map

The `.rotate` method applies a rotation in the image plane, i.e. about an axis out of the page. In the case where we do not specify an angle (or rotation matrix), the image will be rotated such that the world and pixel axes are aligned. In the case of an image in helioprojective coordinate system, this means that solar north will be aligned with the y-like pixel axis of the image

In [ ]:
eui_map_rot = eui_map.rotate(missing=eui_map.min())

In [ ]:
eui_map_rot.plot()

In [ ]:
aia_map_rot = aia_map.rotate(angle=30*u.deg, missing=aia_map.min())

In [ ]:
aia_map_rot.plot(clip_interval=[1, 99.9]*u.percent)

# Crop a map

We commonly want to pare down our full field-of-view to a particular region of interest.
With a map, we can do this using the `submap` method.

We can specify the region of our submap using world coordinates as specified by a `SkyCoord`.
These coordinates can be specified in different coordinate systems and still should work (e.g. helioprojective or heliograhic stonyhurst)

In [ ]:
bottom_left = SkyCoord(-300*u.arcsec, 20*u.arcsec, frame=aia_map.coordinate_frame)
top_right = SkyCoord(390*u.arcsec, 650*u.arcsec, frame=aia_map.coordinate_frame)

In [ ]:
submap = aia_map.submap(bottom_left, top_right=top_right)

In [ ]:
fig = plt.figure()
submap.plot(clip_interval=[1, 99.9]*u.percent)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection=eui_map)
eui_map.plot()
eui_map.draw_quadrangle(bottom_left, top_right=top_right, edgecolor='b')

In [ ]:
fig = plt.figure(figsize=(11, 5))
ax1 = fig.add_subplot(1,2,1,projection=aia_map)
aia_map.plot(axes=ax1, clip_interval=(0.1, 99.99)*u.percent)
# draw rectangle on the map
aia_map.draw_quadrangle(bottom_left, top_right=top_right, 
                        axes=ax1)

ax2 = fig.add_subplot(1,2,2,projection=submap)
submap.plot(clip_interval=(0.5, 99.95)*u.percent)

## WCS axes and plotting

SunPy map uses the [`astropy.visualization.wcsaxes`](https://docs.astropy.org/en/stable/visualization/wcsaxes/index.html#module-astropy.visualization.wcsaxes) module to represent world coordinates. 

Using WCSAxes is very powerful but has important concepts to think about:

 * **`world`** coordinates refer to the coordinates of the coordinate system - i.e. arcsec, degrees!
 * **`pixel`** coordinates refer to the array index of the data! i.e. data[10] etc
 
 
When plotting on WCSAxes it will by default plot in pixel coordinates, you can override this behavior and plot in `world` coordinates by getting the transformation from the axes with `ax.get_transform('world')`. We will use some of these examples below. Its also important to note that when using the `world` coordinates these have to be in **degrees** so make sure to convert arcsec's to degrees.



In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.subplot(projection=aia_map)  

# plot the map
aia_map.plot(clip_interval=[0.5, 99.99]*u.percent)
aia_map.draw_limb()
aia_map.draw_grid()

# plot in pixel coordinates
ax.plot(500, 500, marker='o', color="b",  label="Pixel coord")

# plot in world coordinates
ax.plot((500*u.arcsec).to(u.deg), (500*u.arcsec).to(u.deg),
        transform=ax.get_transform('world'), 
        marker='o',color="g", label="World Coord")

ax.legend()



You can also plot SkyCoords on a Map. Importantly, this can be done with ax.plot_coord and the coordinate does not need to be transformed to the same coordinate frame as the map - it is done automatically if it can be



In [ ]:
coord1 = SkyCoord(200*u.arcsec, -500*u.arcsec, frame=aia_map.coordinate_frame)
coord2 = SkyCoord(20*u.deg, 30*u.deg, frame=frames.HeliographicStonyhurst)

In [ ]:
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(projection=aia_map)
aia_map.plot(axes=ax, clip_interval=[0.5, 99.99]*u.percent)

aia_map.draw_grid(axes=ax)

ax.plot_coord(coord1, marker='o', ms=10, color='b')
ax.plot_coord(coord2, marker='x', ms=10, color='b')



# 3. Coordinates

SunPy uses [`astropy.coordinates`](https://docs.astropy.org/en/stable/coordinates/index.html) to represent points in physical space. This applies to both points in 3D space and projected coordinates in images.

The `sunpy.coordinates` sub-package contains:

* A robust framework for working with solar-physics coordinate systems
* Functions to obtain the locations of solar-system bodies (`sunpy.coordinates.ephemeris`)
* Functions to calculate Sun-specific coordinate information (`sunpy.coordinates.sun`)


In this notebook we'll introduce some of the powerful functionality available within sunpy

## Coordinate frameword
`sunpy` extends the `astropy.coordinates` framework by adding additional solar-specific coordinate frames and the accompanying transformations between them.

<div>
<img src="./images/coordinates.svg"/>
</div>

### Creating coordinates 

We deal with coordinates by using astropy's [`SkyCoord`](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord) class. 

We have already seem some introduction to this in the previous notebook but lets extend this here!


A **coordinate** combines position data with a coordinate frame, and a SkyCoord object is created by passing in positions with specified units and a coordinate frame. Above in the imports cell we've imported [`sunpy.coordinates.frames`](https://docs.sunpy.org/en/stable/code_ref/coordinates/index.html#supported-coordinate-systems) which allow us to use solar physics specific frames such as Helioprojective, Heliographic Stonyhurst, Heliocentric etc. 

Lets create a point on the Sun in lat and long in the Heliographic Stonyhurst coordinate system

In [ ]:
# longitude, latitude
hgs_coord = SkyCoord(10*u.deg, 20*u.deg, obstime="2017-08-01", frame=frames.HeliographicStonyhurst)  
hgs_coord

In [ ]:
hgs_coord_xyz = SkyCoord(hgs_coord, representation_type='cartesian')
hgs_coord_xyz

We can then transform this coordinate to the any defined coordinate frame defined in astropy or sunpy. Lets transform it to the Helioprojective frame (which is observer-based)

In [ ]:
hgs_coord.transform_to(frames.Helioprojective(observer="earth"))

We can also convert this to other coordinate systems outside the solar-specific ones - for examples ICRS

In [ ]:
hgs_coord.transform_to("icrs")

## An important note about observer based frames

Some coordinate frames are defined based on the position of the observer e.g. the Helioprojective and Heliocentric frames. Hence it's important to think about this - particularly when transforming points between coordinate systems. 
This is shown above when transforming to Helioprojective we needed to pass an `observer` keyword. Similarly, its important that the `obstime` is given also!

For example, lets define a point on the Sun in Helioprojective and see what that equivalent point would be from another observer - say Mars!

In [ ]:
obstime = "2022-05-02 00:00"

In [ ]:
hpc_coord = SkyCoord(0*u.arcsec, 0*u.arcsec, observer="earth", 
                     obstime=obstime, frame=frames.Helioprojective)
hpc_coord

In [ ]:
print(hpc_coord.Tx, hpc_coord.Ty)

In [ ]:
mars_hpc_coord = hpc_coord.transform_to(frames.Helioprojective(observer="mars"))
mars_hpc_coord

In [ ]:
print(mars_hpc_coord.Tx, mars_hpc_coord.Ty)

# Observations from different observer locations

## Example of SDO/AIA and Solar Orbiter/EUI

In [ ]:
fig = plt.figure(figsize=(11, 5))
ax1 = fig.add_subplot(1,2,1,projection=aia_map)
ax2 = fig.add_subplot(1,2,2,projection=eui_map)

aia_map.plot(vmin=0, vmax=200, axes=ax1)
aia_map.draw_grid()
eui_map.plot(axes=ax2)
eui_map.draw_grid()

### Plot the solar limb as seen from EUI on AIA map

We can plot the solar limb as seen from EUI on the AIA map

In [ ]:
fig = plt.figure(figsize=(6, 6))

ax1 = fig.add_subplot(projection=aia_map)
aia_map.plot(axes=ax1, vmin=0, vmax=200)
aia_map.draw_limb(axes=ax1, color='white')
eui_map.draw_limb(axes=ax1, color='blue', label="EUI limb")
plt.legend()

In [ ]:
fig = plt.figure(figsize=(6, 6))

ax1 = fig.add_subplot(projection=eui_map)
eui_map.plot(axes=ax1)
eui_map.draw_limb(axes=ax1, color='white')
aia_map.draw_limb(axes=ax1, color='green', label="AIA limb")
plt.legend()



## The solar flare and eruption is both seen by SDO/AIA and from SolO/EUI, just from two different points of view.

Lets define the point and plot the point of the flare as seen by AIA

In [ ]:
flare_coord_aia = SkyCoord(100*u.arcsec, 350*u.arcsec, frame=aia_map.coordinate_frame)

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection=aia_map)
aia_map.plot(vmin=0, vmax=200)
ax.plot_coord(flare_coord_aia, marker='X', color='b', ms=10)

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection=eui_map)
eui_map.plot()
ax.plot_coord(flare_coord_aia, marker='X', color='b', ms=10)

## Reproject AIA to field of view of Solo

Lets say for this observation, we want to identify what the AIA field of view looks like from the observer of Solar Orbiter. We can do this by using `reproject`.

In [ ]:
aia_map = aia_map.resample((512, 512)*u.pix)

In [ ]:
outshape = (1500, 1500)# aia_map.data.shape
ref_coord = SkyCoord(0*u.arcsec, 0*u.arcsec,
                     frame='helioprojective', obstime=eui_map.date, observer=eui_map.observer_coordinate)

# Create a FITS WCS header for the reference coordinate and frame
header = sunpy.map.make_fitswcs_header(
    outshape,
    ref_coord,
    scale=u.Quantity(aia_map.scale),
)
header['rsun_ref'] = aia_map.meta['rsun_ref']



In [ ]:
outmap = aia_map.reproject_to(header)
outmap.plot_settings = aia_map.plot_settings

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection=outmap)
outmap.plot(vmin=0, vmax=100)
outmap.draw_limb(color='k')
aia_map.draw_limb(color='b')
ax.plot_coord(flare_coord_aia, color='b', marker='X', ms=10)
ax.set_title("AIA from view of Solar Orbiter")

## Reproject to Heliographic Maps

As well as reprojecting to different observers, sunpy maps can also be reprojected to different coordinate frames`

In [ ]:
shape_out = (720, 1440)
frame_out = SkyCoord(0, 0, unit=u.deg,
                     frame="heliographic_stonyhurst",
                     obstime=aia_map.date,
                     rsun=aia_map.coordinate_frame.rsun)
header = sunpy.map.make_fitswcs_header(shape_out,
                                       frame_out,
                                       scale=(360 / shape_out[1],
                                              180 / shape_out[0]) * u.deg / u.pix,
                                       projection_code="CAR")
out_wcs = WCS(header)

In [ ]:
outmap = aia_map.reproject_to(header)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(projection=outmap)
outmap.plot(vmin=0, vmax=200)
aia_map.draw_limb(color='b')
eui_map.draw_limb(color='green')

## Positions of solar system bodies
`sunpy.coordinates` provides functions to obtain the coordinates of solar-system bodies.
The function `get_body_heliographic_stonyhurst` which will return the location of the solar-system body in the `HeliographicStonyhurst` frame.

For other solar-system bodies (e.g., major man-made spacecraft or comets), you can use `get_horizons_coord()`, which queries JPL HORIZONS:

In [ ]:
earth_pos = get_body_heliographic_stonyhurst("earth", "2022-03-28")
mars_pos = get_body_heliographic_stonyhurst("mars", "2022-03-28")

In [ ]:
print(mars_pos)

In [ ]:
solo_pos = get_horizons_coord("solar orbiter", "2022-03-28")

## Plotting positions of spacecraft

Lets plot the positions of different spacecraft over the recent Solar Orbiter perihelion!

In [ ]:
perihelion_time = parse_time("2022-03-26")
perihelion_seq = perihelion_time + np.arange(-30, 30)*u.day



In [ ]:
solo_coord = get_horizons_coord("solar orbiter", perihelion_seq)
psp_coord = get_horizons_coord("psp", perihelion_seq)
sdo_coord = get_horizons_coord("sdo", perihelion_seq)

In [ ]:
solo_coord

In [ ]:
fig = plt.figure(dpi=120)
ax = fig.add_subplot(projection='polar')

# Transform to HGS
psp_coord_hgs = psp_coord[0].heliographic_stonyhurst
solo_coord_hgs = solo_coord[0].heliographic_stonyhurst
sdo_coord_hgs = sdo_coord[0].heliographic_stonyhurst


ax.plot(psp_coord_hgs.lon.to('rad'), psp_coord_hgs.radius,
        '.', markersize=5, label='PSP')
ax.plot(solo_coord_hgs.lon.to('rad'), solo_coord_hgs.radius,
        '.', markersize=5, label='SolO')
ax.plot(sdo_coord_hgs.lon.to('rad'), sdo_coord_hgs.radius,
        '.', markersize=5, label='SDO')


ax.legend(loc='lower right')
ax.set_theta_zero_location("S")
ax.set_title('Positions in Heliographic Stonyhurst (HGS)')

In [ ]:
fig = plt.figure(dpi=120)
ax = fig.add_subplot(projection='polar')

# Transform to HGS
psp_coord_hgs = psp_coord.heliocentricinertial
solo_coord_hgs = solo_coord.heliocentricinertial
sdo_coord_hgs = sdo_coord.heliocentricinertial


ax.plot(psp_coord_hgs.lon.to('rad'), psp_coord_hgs.distance,
        '.', markersize=5, label='PSP')
ax.plot(solo_coord_hgs.lon.to('rad'), solo_coord_hgs.distance,
        '.', markersize=5, label='SolO')
ax.plot(sdo_coord_hgs.lon.to('rad'), sdo_coord_hgs.distance,
        '.', markersize=5, label='SDO')


ax.legend(loc='lower right')
ax.set_theta_zero_location("S")
ax.set_title('Positions in Heliographic Stonyhurst (HGS)')